# Libraries

In [17]:
import sys
pathModulesES = '../sauceforyall/'
sys.path.append(pathModulesES)

**Elasticsearch Query**

In [18]:
from yelpquery import YelpQuery
from pandasticsearch import Select
ye = YelpQuery()

**Machine Learning**

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

**Visualize**

In [20]:
import matplotlib.pyplot as plt
import seaborn as sns

**Others**

In [21]:
import re
import string

**Index name**

In [22]:
index_business = "yelp-business*"
index_review = "yelp-review*"
index_tip = "yelp-tip*"
index_user = "yelp-user*"

# 1. Data Pre-Processing

## 1.1 Load data

Retrieve all the reviews from the last 3 years, this helps reduce the volume of data

# **CHOOSE UR CITY**

In [23]:
city = "Toronto"

In [24]:
res = ye.getBusinessByCity(city, size=500)
businesses = ye.getResultScrolling(res)

ConnectionError: ConnectionError((<urllib3.connection.HTTPConnection object at 0x7f4e84f017d0>, 'Connection to 47.91.72.40 timed out. (connect timeout=10)')) caused by: ConnectTimeoutError((<urllib3.connection.HTTPConnection object at 0x7f4e84f017d0>, 'Connection to 47.91.72.40 timed out. (connect timeout=10)'))

In [10]:
biz_ids = businesses["business_id"].to_list()

In [11]:
firstReview = ye.getMultivalueTerm(index="yelp-review*", term="business_id.keyword", valueArray=biz_ids, exclude=["funny","cool","text","date","useful"])

In [12]:
df_review = ye.getResultScrolling(firstReview)

In [13]:
df_review.head()

,_index,_type,_id,_score,review_id,@timestamp,user_id,@version,stars,business_id
0,yelp-review,_doc,nvlwDXIBXYQmZMVBJA1r,1.0,FNPOMKMwkGdfonZtIO-7Zg,2020-05-13T09:49:12.264Z,yJiV9QNk6HUMQJN5PO6svQ,1,5.0,y-Iw6dZflNix4BdwIyTNGA
1,yelp-review,_doc,n_lwDXIBXYQmZMVBJA1r,1.0,eK3QfGNDcJtSM4w8npFgJw,2020-05-13T09:49:12.264Z,SVC0CajvmYfH5uAq4JnGvg,1,4.0,ejKXczjEkm1gzyRdblQm8w
2,yelp-review,_doc,oPlwDXIBXYQmZMVBJA1r,1.0,gpPZ9ijD9G66BKQ3nje8jg,2020-05-13T09:49:12.264Z,c3DfRslN9pM-34shCBYpzA,1,3.0,WnUttoJffplgWaQGR2J2Xw
3,yelp-review,_doc,pflwDXIBXYQmZMVBJA1r,1.0,QguEVceqX__FyUmvcBXs4A,2020-05-13T09:49:12.264Z,OXqpG22fH9oSPO38toN4Tg,1,5.0,R3FZENdOY_TTKykKsJIuRA
4,yelp-review,_doc,6flwDXIBXYQmZMVBJA11,1.0,WENrmRfzgCeYfZ247dg58A,2020-05-13T09:49:12.306Z,S9no6froaqeisNKc5STyPA,1,4.0,c78Pat78fVUBFPXYeVvbaQ


In [14]:
df_review = df_review.drop("_index", axis = 1)
df_review = df_review.drop("_type", axis = 1)
df_review = df_review.drop("_id", axis = 1)
df_review = df_review.drop("@timestamp", axis = 1)
df_review = df_review.drop("@version", axis = 1)
df_review = df_review.drop("_score", axis = 1)
df_review = df_review.drop("review_id", axis = 1)

In [15]:
import pandas as pd
businesses = businesses[["business_id","categories","city","state","name","is_closed"]]
final_df = pd.merge(df_review,businesses,on='business_id')
###
review_df1 =  pd.read_json(data_path + "cleaned/review_cleaned_2016.json", lines=True)
review_df1 = review_df1[["text","user_id","business_id"]]
review_df2 =  pd.read_json(data_path + "cleaned/review_cleaned_2017.json", lines=True)
review_df2 = review_df2[["text","user_id","business_id"]]
review_df3 =  pd.read_json(data_path + "cleaned/review_cleaned_2018.json", lines=True)
review_df3 = review_df3[["text","user_id","business_id"]]
final_df = pd.merge(final_df,review_df,left_on=['business_id','user_id'], right_on = ['business_id','user_id'])
final_df.head()

,user_id,stars,business_id,categories
0,yJiV9QNk6HUMQJN5PO6svQ,5.0,y-Iw6dZflNix4BdwIyTNGA,"[Restaurants, Food, Breweries, Gastropubs, Pizza]"
1,gPta7HDV8aMNeZ8Q97j0Sw,1.0,y-Iw6dZflNix4BdwIyTNGA,"[Restaurants, Food, Breweries, Gastropubs, Pizza]"
2,h8RovWOyC4FvFxrtaBQqJQ,2.0,y-Iw6dZflNix4BdwIyTNGA,"[Restaurants, Food, Breweries, Gastropubs, Pizza]"
3,eSc8pCT0H1ThVV89dfq6Qw,4.0,y-Iw6dZflNix4BdwIyTNGA,"[Restaurants, Food, Breweries, Gastropubs, Pizza]"
4,fM9LC2P8jQrQGGsXKccLQw,4.0,y-Iw6dZflNix4BdwIyTNGA,"[Restaurants, Food, Breweries, Gastropubs, Pizza]"


In [16]:
final_df.to_json("../dataset/cleaned/review_stars_"+city+".json", orient='records', lines=True)